In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests

the median stake calcualtion should be applied to all the contested validators, who are eligible for elected next epoch.

- before election, get the top 200 eligible nodes, and calculate the median;
- after election, calculate the median for the elected node. both the two status:
    - currently elected and signing enough blocks to be eligible for election next epoch
    - currently elected and not signing enough blocks to be eligible for election next epoch

challenge: in the same epoch, the delegation number will change, needs to get the median before the election (the last block in one epoch). However, when validator number increases, it's hard to get all the validator's stakes in 8 seconds (one block time)

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result']

In [6]:
def getEposMedian():
    method = "hmy_getMedianRawStakeSnapshot"
    params = []
    return float(get_information(method, params)['result']['epos-median-stake'])

In [7]:
def get_median(lst):
    n = len(lst) 
    lst.sort() 
    if n % 2 == 0: 
        median1 = lst[n//2] 
        median2 = lst[n//2 - 1] 
        median = (median1 + median2)/2
    else: 
        median = lst[n//2] 
    return median

In [8]:
def M3_test():
    print("Test-M3: Median function stability: run median computation for x number of epoch to verify stability")
    num = int(input("enter the number of epoches you want to test: "))
    # reach the last block in current epoch to get the eligible validator address
    
    iterations = 0
    while iterations < num:
        print(str(iterations+1) + "iteration will begin ...")
        block = getBlockNumber()
        last_block = getLastBlockOfCurrentEpoch()
        print("current and last block numbers", block, last_block)
        print("wait until the new epoch begins at block number ...", last_block + 1)
        block = getBlockNumber()
        while block < last_block:
            block = getBlockNumber()

        new_epoch_block = block + 1
        while block < new_epoch_block:
            block = getBlockNumber()
        print("new epoch first block reached", new_epoch_block, "will begin testing")

        # get the median from rpc call
        median = getEposMedian()
        # calculate the median manually
        validator_infos = getAllValidatorInformation()
        stake = []
        for i in validator_infos:
            stake.append(i['total-delegation'])
        cal_median = float(get_median(stake))
        # compare the calculated median and rpc median
        if cal_median != median:
            print("Test-M3: Fail")
            print("manually calculated median stake: " + str(cal_median))
            print("harmony apr call median stake: " + str(median))
        else:
            print("Test-M3: Success")
        iterations += 1    

In [10]:
M3_test()

Test-M3: Median function stability: run median computation for x number of epoch to verify stability
enter the number of epoches you want to test: 2
1iteration will begin ...
current and last block numbers 5514 5547
wait until the new epoch begins at block number ... 5548
new epoch first block reached 5548 will begin testing
Test-M3: Fail
manually calculated median stake: 2.1e+23
harmony apr call median stake: 2.04944e+23
2iteration will begin ...
current and last block numbers 5548 5585
wait until the new epoch begins at block number ... 5586
new epoch first block reached 5586 will begin testing
Test-M3: Fail
manually calculated median stake: 2.079995e+23
harmony apr call median stake: 1.99888e+23
